In [1]:
import pandas as pd
d = pd.read_csv("C:/Users/dridi/Downloads/Youtube01-Psy.csv")

In [2]:
d.tail(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
340,z12exzcrvpeew1yxg04cd5tbwnmfubnh4kk0k,Anthony1SV,2014-11-14T00:01:37,Please do buy these new Christmas shirts! You ...,1
341,z13qyxk5tzq1e5asx22xjt3wdq3ns32f5,Ameenk Chanel,2014-11-14T11:50:02,Free my apps get 1m crdits ! Just click on the...,1
342,z12uyzhazxqkzbrzz04cedqpkwzdvjyy3u40k,Brandon Wilson,2014-11-14T12:21:52,Why does a song like this have more views than...,0
343,z13sh3cz1kbqgrai504cf53qsq25ypmi5zs0k,Leonel Hernandez,2014-11-14T12:35:38,"Something to dance to, even if your sad JUST ...",0
344,z12etrpq5xu0vnm2g230szo5ote3zviny,InterGaming,2014-11-14T13:16:24,everyones back lool this is almost 3 years old...,0
345,z13th1q4yzihf1bll23qxzpjeujterydj,Carmen Racasanu,2014-11-14T13:27:52,How can this have 2 billion views when there's...,0
346,z13fcn1wfpb5e51xe04chdxakpzgchyaxzo0k,diego mogrovejo,2014-11-14T13:28:08,I don't now why I'm watching this in 2014﻿,0
347,z130zd5b3titudkoe04ccbeohojxuzppvbg,BlueYetiPlayz -Call Of Duty and More,2015-05-23T13:04:32,subscribe to me for call of duty vids and give...,1
348,z12he50arvrkivl5u04cctawgxzkjfsjcc4,Photo Editor,2015-06-05T14:14:48,hi guys please my android photo editor downloa...,1
349,z13vhvu54u3ewpp5h04ccb4zuoardrmjlyk0k,Ray Benich,2015-06-05T18:05:16,The first billion viewed this because they tho...,0


In [3]:
len(d.query('CLASS == 1'))

175

In [4]:
len(d.query('CLASS == 0'))

175

In [5]:
len(d)

350

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [7]:
dvec = vectorizer.fit_transform(d['CONTENT'])

In [8]:
dvec

<350x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 4354 stored elements in Compressed Sparse Row format>

In [9]:
analyze = vectorizer.build_analyzer()

In [10]:
print(d['CONTENT'][349])
analyze(d['CONTENT'][349])

The first billion viewed this because they thought it was really cool, the  other billion and a half came to see how stupid the first billion were...﻿


['the',
 'first',
 'billion',
 'viewed',
 'this',
 'because',
 'they',
 'thought',
 'it',
 'was',
 'really',
 'cool',
 'the',
 'other',
 'billion',
 'and',
 'half',
 'came',
 'to',
 'see',
 'how',
 'stupid',
 'the',
 'first',
 'billion',
 'were']

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_train = ['the',
 'first',
 'billion',
 'viewed',
 'this',
 'because',
 'they',
 'thought',
 'it',
 'was',
 'really',
 'cool',
 'the',
 'other',
 'billion',
 'and',
 'half',
 'came',
 'to',
 'see',
 'how',
 'stupid',
 'the',
 'first',
 'billion',
 'were']
vectorizer.fit(X_train)

feature_names = vectorizer.get_feature_names()
print(feature_names)


['and', 'because', 'billion', 'came', 'cool', 'first', 'half', 'how', 'it', 'other', 'really', 'see', 'stupid', 'the', 'they', 'this', 'thought', 'to', 'viewed', 'was', 'were']


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
dshuf = d.sample(frac=1)

In [13]:
d_train = dshuf[:300]
d_test = dshuf[300:]
d_train_att = vectorizer.fit_transform(d_train['CONTENT']) # fit bag-of-words on training set
d_test_att = vectorizer.transform(d_test['CONTENT']) # reuse on testing set
d_train_label = d_train['CLASS']
d_test_label = d_test['CLASS']

In [14]:
d_train_att

<300x1303 sparse matrix of type '<class 'numpy.int64'>'
	with 3792 stored elements in Compressed Sparse Row format>

In [15]:
d_test_att

<50x1303 sparse matrix of type '<class 'numpy.int64'>'
	with 443 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80)

In [17]:
clf.fit(d_train_att, d_train_label)

RandomForestClassifier(n_estimators=80)

In [18]:
clf.score(d_test_att, d_test_label)

1.0

In [19]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(d_test_att)
confusion_matrix(d_test_label, pred_labels)

array([[25,  0],
       [ 0, 25]], dtype=int64)

In [20]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, d_train_att, d_train_label, cv=5)
# show average score and +/- two standard deviations away (covering 95% of scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.03)


In [21]:
# load all datasets and combine them
d = pd.concat([pd.read_csv("C:/Users/dridi/Downloads/Youtube01-Psy.csv"),
               pd.read_csv("C:/Users/dridi/Downloads/Youtube02-KatyPerry.csv"),
               pd.read_csv("C:/Users/dridi/Downloads/Youtube03-LMFAO.csv"),
               pd.read_csv("C:/Users/dridi/Downloads/Youtube04-Eminem.csv"),
               pd.read_csv("C:/Users/dridi/Downloads/Youtube05-Shakira.csv")])


In [22]:
len(d)

1956

In [23]:
len(d.query('CLASS == 1'))

1005

In [24]:
len(d.query('CLASS == 0'))

951

In [25]:
dshuf = d.sample(frac=1)
d_content = dshuf['CONTENT']
d_label = dshuf['CLASS']

In [26]:
# set up a pipeline
from sklearn.pipeline import Pipeline, make_pipeline
pipeline = Pipeline([
    ('bag-of-words', CountVectorizer()),
    ('random forest', RandomForestClassifier()),
])
pipeline

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [27]:
# or: pipeline = make_pipeline(CountVectorizer(), RandomForestClassifier())
make_pipeline(CountVectorizer(), RandomForestClassifier())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [28]:
pipeline.fit(d_content[:1500],d_label[:1500])

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [29]:
pipeline.score(d_content[15:], d_label[15:])

0.9876352395672334

In [30]:
pipeline.predict(["what a neat video!"])

array([0], dtype=int64)

In [31]:
pipeline.predict(["plz subscribe to my channel"])

array([1], dtype=int64)

In [32]:
scores = cross_val_score(pipeline, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.02)


In [33]:
# add tfidf
from sklearn.feature_extraction.text import TfidfTransformer
pipeline2 = make_pipeline(CountVectorizer(),
                          TfidfTransformer(norm=None),
                          RandomForestClassifier())

In [34]:
scores = cross_val_score(pipeline2, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.03)


In [35]:
pipeline2.steps

[('countvectorizer', CountVectorizer()),
 ('tfidftransformer', TfidfTransformer(norm=None)),
 ('randomforestclassifier', RandomForestClassifier())]

In [36]:
# parameter search
parameters = {
    'countvectorizer__max_features': (None, 1000, 2000),
    'countvectorizer__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'countvectorizer__stop_words': ('english', None),
    'tfidftransformer__use_idf': (True, False), # effectively turn on/off tfidf
    'randomforestclassifier__n_estimators': (20, 50, 100)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline2, parameters, n_jobs=-1, verbose=1)

In [37]:
grid_search.fit(d_content, d_label)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer',
                                        TfidfTransformer(norm=None)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 1), (1, 2)),
                         'countvectorizer__stop_words': ('english', None),
                         'randomforestclassifier__n_estimators': (20, 50, 100),
                         'tfidftransformer__use_idf': (True, False)},
             verbose=1)

In [38]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.961
Best parameters set:
	countvectorizer__max_features: 2000
	countvectorizer__ngram_range: (1, 1)
	countvectorizer__stop_words: 'english'
	randomforestclassifier__n_estimators: 50
	tfidftransformer__use_idf: True
